In [1]:
include("distances/new_distance.jl")
include("distances/distance_Wasserstein.jl")
using RCall # to call R functions



In [2]:
R"""

library(transport)

# Measures are given as a nTop x nBottom x 2 array
# nTop = n in the article 
# nBottom = m in the article 
# nGrid = M in the article 
# a[,,1] -> location of the atom 
# a[,,2] -> mass of the atom 

# First simplest 1D Wasserstein distance between measures with the same number of atom and equal measures

wasserstein1DUniform <- function(atoms1, atoms2, p) {
  # atoms1 and atoms2 only list of position 
  # p is the exponent 
  if (length(atoms1) == length(atoms2)) {

    diff = abs(sort(atoms1) - sort(atoms2))
    return(mean((diff^p))^(1/p))
  } else {
    print("ERROR: not the same number of atoms")
    return(-1.0)
  }
}


ww <- function(measure1, measure2, p) {
  # Assuming that the number of atoms at the lower level is the same in each measure 
  
  s1 <- dim(measure1)
  s2 <- dim(measure2)
  
  if (s1[2] != s2[2]) {
    cat("PROBLEM OF DIMENSION: each lower measure should have the same dimension\n")
    return(-1.0)
  } else {
    

    # Extract dimensions
    m1 <- s1[1]
    m2 <- s2[1]
    n  <- s1[2]
    
    # Compute matrix of pairwise distances which will be a cost function 
    
    C <- matrix(0, m1, m2)
    for (i in 1:m1) {
      for (j in 1:m2) {
        C[i, j] <- wasserstein1DUniform(measure1[i, ], measure2[j, ], p)^p
      }
    }
    

    # Build the weights: uniform 
    weight1 <- rep(1 / m1, m1)
    weight2 <- rep(1 / m2, m2)
    
    # Solving the optimal transport problem 
    output <-(wasserstein(weight1, weight2, p = 1, costm = C, prob = TRUE))^(1 / p)
    return(output)
  }
}


"""


RObject{ClosSxp}
function (measure1, measure2, p) 
{
    s1 <- dim(measure1)
    s2 <- dim(measure2)
    if (s1[2] != s2[2]) {
        cat("PROBLEM OF DIMENSION: each lower measure should have the same dimension\n")
        return(-1)
    }
    else {
        m1 <- s1[1]
        m2 <- s2[1]
        n <- s1[2]
        C <- matrix(0, m1, m2)
        for (i in 1:m1) {
            for (j in 1:m2) {
                C[i, j] <- wasserstein1DUniform(measure1[i, ], 
                  measure2[j, ], p)^p
            }
        }
        weight1 <- rep(1/m1, m1)
        weight2 <- rep(1/m2, m2)
        output <- (wasserstein(weight1, weight2, p = 1, costm = C, 
            prob = TRUE))^(1/p)
        return(output)
    }
}


In [3]:
R""" 




lowerBound <- function(measure1, measure2) {
  s1 <- dim(measure1)
  s2 <- dim(measure2)
  
  if (s1[1] != s2[1]) {
    message("PROBLEM OF DIMENSION: should have measures with the same dimensions")
    return(-1.0)
  } else {
    nTop <- s1[1]
    toTransport1 <- rowSums(measure1[, , 1] * measure1[, , 2])
    toTransport2 <- rowSums(measure2[, , 1] * measure2[, , 2])
    return( (1 / nTop) * sum(abs(sort(toTransport1) - sort(toTransport2))) )
  }
}

# --------------------------------------------------------------
# Auxiliary functions for the new distance
# --------------------------------------------------------------

projectionGrid <- function(x, a, b, nGrid) {
  if ((x < a) | (x > b)) {
    message("PROBLEM with the projection: wrong bounds")
    return(-1)
  } else {
    # Julia: round(Int, (x - a)/(b - a) * nGrid) + 1
    return(as.integer(round((x - a) / (b - a) * nGrid)) + 1)
  }
}

projectionGridMeasure <- function(measure, a, b, nGrid) {
  output <- numeric(nGrid + 1)
  nBottom <- dim(measure)[1]
  for (i in 1:nBottom) {
    idx <- projectionGrid(measure[i, 1], a, b, nGrid)
    output[idx] <- output[idx] + measure[i, 2]
  }
  return(output)
}

buildEvaluationMatrixGrid <- function(a, b, nGrid) {
  deltaX <- (b - a) / nGrid
  output <- matrix(0, nrow = nGrid + 1, ncol = nGrid)
  for (i in 1:nGrid) {
    output[i + 1, 1:i] <- rep(deltaX, i)
  }
  return(output)
}

evaluationObjectiveGrid <- function(unknown, weights, Q, derivative) {
  # derivative: TRUE or FALSE, to decide if to output the derivative
  
  nTop <- dim(weights)[2]
  nGrid <- dim(Q)[2]
  
  # Value of the function f on grid points:
  # Julia uses: f = vcat(0., cumsum(unknown)) * Q[2,1]
  f <- c(0, cumsum(unknown)) * Q[2, 1]
  
  # Integrals
  integralF <- matrix(0, nrow = 2, ncol = nTop)
  for (k in 1:2) {
    # weights[k, i, ] %*% f  for each i
    for (i in 1:nTop) {
      integralF[k, i] <- sum(weights[k, i, ] * f)
    }
  }
  
  value <- (1 / nTop) * sum(abs(sort(integralF[1, ]) - sort(integralF[2, ])))
  
  if (!isTRUE(derivative)) {
    return(list(value = value, grad = 0.0))
  } else {
    perm1 <- order(integralF[1, ])
    perm2 <- order(integralF[2, ])
    
    grad <- numeric(nGrid)
    
    Qt <- t(Q)  # (nGrid) x (nGrid+1)
    for (i in 1:nTop) {
      s <- sign(integralF[1, perm1[i]] - integralF[2, perm2[i]])
      diff_w <- weights[1, perm1[i], ] - weights[2, perm2[i], ]  # length nGrid+1
      grad <- grad + s * (1 / nTop) * as.vector(Qt %*% diff_w)
    }
    
    return(list(value = value, grad = grad))
  }
}

# --------------------------------------------------------------
# HIPM distance (dlip)
# --------------------------------------------------------------

dlip <- function(measure1, measure2, a, b, nGrid = 250, nSteps = 1000,
                 nRerun = 5, tol = 1e-4) {
  
  s1 <- dim(measure1)
  s2 <- dim(measure2)
  
  if (s1[1] != s2[1]) {
    message("PROBLEM OF DIMENSION: should have measures with the same dimensions")
    return(-1.0)
  } else {
    nTop <- s1[1]
    deltaX <- (b - a) / nGrid
    
    # Project atoms on the grid
    weightsAtoms <- array(0, dim = c(2, nTop, nGrid + 1))# First coordinate is for measure1/measure2, 
                                    # second for the m index and third is the point of the grid

    for (i in 1:nTop) {
      weightsAtoms[1, i, ] <- projectionGridMeasure(matrix(measure1[i, , ],s1[2],2), a, b, nGrid)
      weightsAtoms[2, i, ] <- projectionGridMeasure(matrix(measure2[i, , ],s2[2],2), a, b, nGrid)
    }
    
    # Build Q
    Q <- buildEvaluationMatrixGrid(a, b, nGrid)
    
    # Gradient ascent (multiple restarts)
    valueFunction <- matrix(0, nrow = nRerun, ncol = nSteps)
    unknownArray  <- matrix(0, nrow = nRerun, ncol = nGrid)
    
    for (k in 1:nRerun) {
      if (k == 1L) {
        unknown <- rep(1.0, nGrid)
      } else {
        coeff <- runif(3, min = -1, max = 1)
        xs <- seq(0, pi, length.out = nGrid)
        unknown <- (coeff[1] / 2) * cos(xs) +
          (coeff[2] / 4) * cos(2 * xs) +
          (coeff[3] / 8) * cos(3 * xs)
        unknown <- pmax(pmin(unknown, 1), -1)  # clamp to [-1,1]
      }
      
      i <- 1
      continueLoop <- TRUE
      
      while ((i <= nSteps) && continueLoop) {
        eval <- evaluationObjectiveGrid(unknown, weightsAtoms, Q, derivative = TRUE)
        f  <- eval$value
        df <- eval$grad
        
        ascentDirection <- df
        t_max <- 1 / tol
        
        # Project ascent direction onto the admissible set and find max step
        for (j in 1:nGrid) {
          if (unknown[j] > 1 - tol) {
            unknown[j] <- 1
            if (df[j] > -tol) {
              ascentDirection[j] <- 0
            } else {
              t_max <- min(((-1 - unknown[j]) / ascentDirection[j]), t_max)
            }
          } else if (unknown[j] < -1 + tol) {
            unknown[j] <- -1
            if (df[j] < tol) {
              ascentDirection[j] <- 0
            } else {
              t_max <- min(((1 - unknown[j]) / ascentDirection[j]), t_max)
            }
          } else {
            if (ascentDirection[j] > tol) {
              t_max <- min(((1 - unknown[j]) / ascentDirection[j]), t_max)
            } else if (ascentDirection[j] < -tol) {
              t_max <- min(((-1 - unknown[j]) / ascentDirection[j]), t_max)
            }
          }
        }
        
        # Move only if predicted ascent is significant
        predicted <- sum(df * ascentDirection) * deltaX
        if (predicted < 1e-8) {
          candidateUnknown <- unknown
          newf <- f
          continueLoop <- FALSE
        } else {
          expectedIncrease <- sum(df * ascentDirection)
          t <- t_max
          candidateUnknown <- unknown + t * ascentDirection
          newf <- evaluationObjectiveGrid(candidateUnknown, weightsAtoms, Q, derivative = FALSE)$value
          
          while ((newf < f + 0.5 * t * expectedIncrease) && (t > 1/128)) {
            t <- t * 0.5
            candidateUnknown <- unknown + t * ascentDirection
            newf <- evaluationObjectiveGrid(candidateUnknown, weightsAtoms, Q, derivative = FALSE)$value
          }
          
          # Stop if increase is too small
          if (newf - f <= sqrt(sum(df^2)) * sqrt(deltaX) * tol^2) {
            continueLoop <- FALSE
          }
        }
        
        unknown <- candidateUnknown
        valueFunction[k, i] <- newf
        i <- i + 1L
      }
      
      # Fill remaining steps with last value (for plotting consistency)
      if (i < (nSteps + 1L)) {
        valueFunction[k, i:nSteps] <- valueFunction[k, i - 1L]
      }
      
      unknownArray[k, ] <- unknown
    }
    
    # Best run
    idx <- which.max(valueFunction[, nSteps])
    return(max(valueFunction[idx, nSteps]))
    # If you also want the optimal function on grid:
    # return(list(value = max(valueFunction[idx, nSteps]),
    #             f = buildEvaluationMatrixGrid(a, b, nGrid) %*% unknownArray[idx, ]))
  }
}



"""


RObject{ClosSxp}
function (measure1, measure2, a, b, nGrid = 250, nSteps = 1000, 
    nRerun = 5, tol = 1e-04) 
{
    s1 <- dim(measure1)
    s2 <- dim(measure2)
    if (s1[1] != s2[1]) {
        message("PROBLEM OF DIMENSION: should have measures with the same dimensions")
        return(-1)
    }
    else {
        nTop <- s1[1]
        deltaX <- (b - a)/nGrid
        weightsAtoms <- array(0, dim = c(2, nTop, nGrid + 1))
        for (i in 1:nTop) {
            weightsAtoms[1, i, ] <- projectionGridMeasure(matrix(measure1[i, 
                , ], s1[2], 2), a, b, nGrid)
            weightsAtoms[2, i, ] <- projectionGridMeasure(matrix(measure2[i, 
                , ], s2[2], 2), a, b, nGrid)
        }
        Q <- buildEvaluationMatrixGrid(a, b, nGrid)
        valueFunction <- matrix(0, nrow = nRerun, ncol = nSteps)
        unknownArray <- matrix(0, nrow = nRerun, ncol = nGrid)
        for (k in 1:nRerun) {
            if (k == 1L) {
                unknown <- rep(1, nGrid)
           

In [4]:
function ww_from_r(measure1::Matrix{Float64}, measure2::Matrix{Float64}, p::Int);
    # This function computes Wasserstein distance between two samples of Normal distributions using R.

                         


    @rput measure1 measure2 p

    R"""
    # if (!requireNamespace("frechet", quietly = TRUE)) {
    #   install.packages("frechet", repos="https://cloud.r-project.org")
    # }
    dist_ww = ww(measure1, measure2, p)
    """
    @rget dist_ww

    return dist_ww
end

function dlip_from_r(measure1::Array{Float64,3}, measure2::Array{Float64,3}, a::Float64, b::Float64);
    # This function computes Wasserstein distance between two samples of Normal distributions using R.

                         


    @rput measure1 measure2 a b

    R"""
    # if (!requireNamespace("frechet", quietly = TRUE)) {
    #   install.packages("frechet", repos="https://cloud.r-project.org")
    # }
    hipm = dlip(measure1, measure2, a, b)
    """
    @rget hipm

    return hipm
end













dlip_from_r (generic function with 1 method)

In [10]:
n = 30
m = 1


atoms_1 = randn(n, m)
atoms_2 = randn(n, m)

a = minimum([minimum(atoms_1), minimum(atoms_2)])
b = maximum([maximum(atoms_1), maximum(atoms_2)])

measure_1 = zeros(n,m,2)
measure_2 = zeros(n,m,2)

measure_1[:,:,1] = atoms_1
measure_1[:,:,2] = fill(1/m, (n,m))

measure_2[:,:,1] = atoms_2
measure_2[:,:,2] = fill(1/m, (n,m))



p = 1

dist_ww_julia = ww(atoms_1, atoms_2, p)
dist_ww_r = ww_from_r(atoms_1, atoms_2, p)
dist_hipm_julia = dlip(measure_1, measure_2, a, b)
dist_hipm_r = dlip_from_r(measure_1, measure_2, a, b)

println("Wasserstein distance from R: ", dist_ww_r)
println("Wasserstein distance from Julia: ", dist_ww_julia)
println("dlipn distance from Julia: ", dist_hipm_julia)
println("dlipn distance from R: ", dist_hipm_r)




Wasserstein distance from R: 0.2319175333524233
Wasserstein distance from Julia: 0.23191702389676527
dlipn distance from Julia: 0.23015894004782103
dlipn distance from R: 0.23015894004782111
